<a href="https://colab.research.google.com/github/RussAbbott/Wyrdl/blob/main/Wyrdl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# This is the start of a longer word list.

wordle_word_list = [
    'aback', 'abase', 'abate', 'abbey', 'abbot', 'abhor', 'abide', 'abled', 'abode', 'abort',
    'about', 'above', 'abuse', 'abyss', 'acorn', 'acrid', 'actor', 'acute', 'adage', 'adapt',
    'adept', 'adieu', 'admin', 'admit', 'adobe', 'adopt', 'adore', 'adorn', 'adult', 'affix',
    'afire', 'afoot', 'afoul', 'after', 'again', 'agape', 'agate', 'agent', 'agile', 'aging',
    'aglow', 'agony', 'agora', 'agree', 'ahead', 'aider', 'aisle', 'alarm', 'album', 'alert',
    'algae', 'alibi', 'alien', 'align', 'alike', 'alive', 'allay', 'alley', 'allot', 'allow',
    'alloy', 'aloft', 'alone', 'along', 'aloof', 'aloud', 'alpha', 'altar', 'alter', 'amass',
    'amaze', 'amber', 'amble', 'amend', 'amiss', 'amity', 'among', 'ample', 'amply', 'amuse',
    'angel', 'anger', 'angle', 'angry', 'angst', 'anime', 'ankle', 'annex', 'annoy', 'annul',
    'anode', 'antic', 'anvil', 'aorta', 'apart', 'aphid', 'aping', 'apnea', 'apple', 'apply',
    'apron', 'aptly', 'arbor', 'ardor', 'arena', 'argue', 'arise', 'armor', 'aroma', 'arose',
    'array', 'arrow', 'arson', 'artsy', 'ascot', 'ashen', 'aside', 'askew', 'assay', 'asset',
    'atoll', 'atone', 'attic', 'audio', 'audit', 'augur', 'aunty', 'avail', 'avert', 'avian',
    'avoid', 'await', 'awake', 'award', 'aware', 'awash', 'awful', 'awoke', 'axial', 'axiom',
    'axion', 'azure']


# ------------------- wyrdl.py -------------------

import contextlib
import pathlib
import random
from rich.console import Console
from rich.theme import Theme
from string import ascii_letters, ascii_uppercase
from typing import List, Tuple, Dict


WYRDLE_THEME = Theme({'correct': 'bold black on #ddffdd',
                      'incorrect': 'bold black on #ffdddd',
                      "warning": "red on #eeffee",
                      })

console = Console(width=80, theme=WYRDLE_THEME)

NUM_LETTERS = 5
NUM_GUESSES = 6
# WORDS_PATH = pathlib.Path(__file__).parent / "wordlist.txt"
WORDS_LIST = wordle_word_list

# STYLES maps scores to styles. E.g., the score 'g' (for 'green') is mapped to the style "bold black on #00FF00".
# The score '_', which means no information (yet) about this letter, is mapped to 'dim'.
# Wouldn't help to add these to the Theme since a Theme style can only be applied to a complete print statement,
# not to a segment.
STYLES = {'g': "bold black on #00ff00",
          'y': "bold black on #ffff00",
          '#': "bold white on #000000",
          '_': "dim",
          }

# SCORE_RANKING assigns an integer to each score so that they can be compared with '>'.
# The ordering is '_#yg'.
SCORE_RANKING: Dict[str, int] = {score: rank for rank, score in enumerate('_#yg')}

verbose: bool

def main(verbo=False):
    global verbose
    verbose = verbo

    # Pre-process: get the secret word and set up lists and a dictionary.
    # secret_word = get_random_word(WORDS_PATH.read_text(encoding="utf-8").split("\n"))
    secret_word = get_random_word(WORDS_LIST)
    if verbose:
        print(secret_word)

    # letter_to_score_map is a dictionary of letters mapped to scores.
    # The scores are updated as new information is found. Initially, we
    # have no information about any letter.
    letter_to_score_map: Dict[str, str] = {letter: '_' for letter in ascii_uppercase}

    # guesses is a list of the guesses
    guesses = []
    # scores is a list of the scores. Must keep track of the scores so that when we display
    # the sequences of guesses, earlier guesses will have the score valid at the time.
    scores = []

    # Main loop: make the guesses
    found_it = False
    with contextlib.suppress(KeyboardInterrupt):
        for idx in range(NUM_GUESSES):
            refresh_page(headline=f"Guess {idx + 1}")

            # Make and score a guess and update guesses, scores, and letter_to_score_map
            guesses, scores, letter_to_score_map = take_a_guess(secret_word, guesses, scores, letter_to_score_map)
            show_guesses_and_letters(guesses, scores, letter_to_score_map)
            if found_it := guesses[-1] == secret_word:
                break

    # Post-process: report the result
    game_over(secret_word, found_it)

def game_over(secret_word, found_it):
    (correct_sorry, is_was, style) = ('Correct', 'is',  'correct') if found_it else \
                                     ('Sorry',   'was', 'incorrect')
    console.print(f"\n{correct_sorry}, the secret word {is_was} {secret_word}", style=style)

def get_random_word(word_list):
    if words := [word.upper() for word in word_list
                    if len(word) == NUM_LETTERS and
                       all(letter in ascii_letters for letter in word)]:
        return random.choice(words)
    else:
        console.print(
            f"No words of length {NUM_LETTERS} in the word list",
            style="warning",
        )
        raise SystemExit()

def guess_a_word(previous_guesses) -> str:
    guess = console.input("\nGuess word: ").upper()

    if guess in previous_guesses:
        console.print(f"You've already guessed {guess}.", style="warning")
        return guess_a_word(previous_guesses)

    if len(guess) != NUM_LETTERS:
        console.print(
            f"Your guess must be {NUM_LETTERS} letters.", style="warning"
        )
        return guess_a_word(previous_guesses)

    if any((invalid := letter) not in ascii_letters for letter in guess):
        console.print(
            f"Invalid letter: '{invalid}'. Please use English letters.", style="warning",
        )
        return guess_a_word(previous_guesses)

    return guess

def refresh_page(headline):
    console.clear()
    console.rule(f"[#00CCFF]:mage: {headline} :bulb:[/]\n")

def score_a_guess(secret_word, guess) -> str:
    '''
    Build a list with one char for each guess element.
    g = Letter in Correct position (green);
    y = Letter in Incorrect position (yellow);
    # = Letter not in secret_word. (These are the gray results. Can't use 'g' since we use it for green)

    In the following example: secret_word = 'hpaay'
                              guess       = 'aaapz'
                              -> score    = 'y#gy#'

    The third letter in both the secret_word and the guess is "a".
        So the third letter in the score must be "g".
        (The matching letters must be determined first.)
    The first letter in the guess, 'a', is associated with the fourth letter in the secret_word, 'a'.
        So the first letter in the score is "y".
    The second letter in the guess, 'a', has nothing in secret_word to be associated with.
        So the second letter in the score is "#".
    The other two letter in the score should be self explanatory.
    '''

    # Generate the score list with the letters in the correct position marked 'g'.
    # All non-green elements are marked '#'. These will be changed to 'y' later if appropriate.
    score = [('g' if sec_ltr == guess_ltr else '#') for sec_ltr, guess_ltr in zip(secret_word, guess)]

    # Generate a list of the "holes," i.e., secret_word letters that do not have correct corresponding guess letters.
    secret_word_holes = [sec_ltr for sec_ltr, score_ltr in zip(secret_word, score) if score_ltr != 'g']

    if verbose: print(f'{secret_word_holes = }')

    # For each available secret_word hole find matching guess letters. Ignore guess letters that are
    # paired with their secret_word letters. Score the first match as 'y' and don't look further.
    for ltr in secret_word_holes:
        for i in range(len(score)):
            # Have we found a hole-filling letter in guess that is not otherwise committed?
            # If so, mark it 'y'. Since we have now filled this hole, go on to the next hole.
            if guess[i] == ltr and score[i] == '#':
                score[i] = 'y'
                break

    # Return score as a string
    score_string = ''.join(score)
    if verbose:
        print(f'\n\t{score_string = }\n\n')
    return score_string

def show_guesses_and_letters(guesses, guess_scores, letter_to_score_map):

    # A letter may appear multiple times in a guess and have different scores.
    def letters_and_scores_to_styled_letters(zipped_letters_scores) -> List[str]:
        return [f"[{STYLES[letter_score]}]{letter}[/]" for letter, letter_score in zipped_letters_scores]

    def spaced_and_styled_seq(zipped_letters_scores) -> str:
        return " ".join(letters_and_scores_to_styled_letters(zipped_letters_scores))

    # Print the sequences of guesses, one styled guess per line.
    for guess, guess_score in zip(guesses, guess_scores):
        console.print(spaced_and_styled_seq(zip(guess, guess_score)), justify="center")

    console.print("\n" + spaced_and_styled_seq(letter_to_score_map.items()) + "\n\n", justify="center")

def take_a_guess(secret_word, guesses, guess_scores, letter_to_score_map) -> Tuple[List[str], List[str], Dict[str, str]]:
    """
    Make and score a new guess. Update and return guesses, guess_scores, and letter_to_score_map.
    """

    guess = guess_a_word(previous_guesses=guesses)
    guess_score = score_a_guess(secret_word, guess)

    # Update letter_to_score_map
    for letter, letter_score in zip(guess, guess_score):
        if SCORE_RANKING[letter_score] > SCORE_RANKING[letter_to_score_map[letter]]:
            letter_to_score_map[letter] = letter_score

    # Use + instead of append() to be more functional.
    # When using +, must put guess into a list since + concatenates its arguments.
    return guesses + [guess], guess_scores + [guess_score], letter_to_score_map

main()

──────────────────────────────── 🧙 Guess 1 💡  ────────────────────────────────

Guess word:

seuio


S E U I O

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z

...

S E U I O

A N G L E

A S K E W

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z

Correct, the secret word is ASKEW